In [2]:
import os
import urllib.request
from datetime import datetime
import pandas as pd

**Автоматичне завантаження csv з урахуванням колізій даних та довантаження**

In [3]:
folder = "vhi"
os.makedirs(folder, exist_ok=True)

url = "https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={}&year1=1981&year2=2024&type=Mean"


def download(reg_index):
    vurl = url.format(reg_index)
    response = urllib.request.urlopen(vurl)
    data = response.read()

    remove_files(reg_index)

    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"VHI_{reg_index}_{timestamp}.csv"
    filepath = os.path.join(folder, filename)

    with open(filepath, 'wb') as file:
        file.write(data)
    print(f"{filename} завантажено")

def remove_files(reg_index):
    for file in os.listdir(folder):  
        if file.startswith(f"VHI_{reg_index}_"):
            os.remove(os.path.join(folder, file))

def data_update():
    for i in range(1, 28):  
        download(i)

data_update()

VHI_1_2025-03-27_12-19-15.csv завантажено
VHI_2_2025-03-27_12-19-16.csv завантажено
VHI_3_2025-03-27_12-19-17.csv завантажено
VHI_4_2025-03-27_12-19-18.csv завантажено
VHI_5_2025-03-27_12-19-19.csv завантажено
VHI_6_2025-03-27_12-19-20.csv завантажено
VHI_7_2025-03-27_12-19-21.csv завантажено
VHI_8_2025-03-27_12-19-22.csv завантажено
VHI_9_2025-03-27_12-19-23.csv завантажено
VHI_10_2025-03-27_12-19-24.csv завантажено
VHI_11_2025-03-27_12-19-25.csv завантажено
VHI_12_2025-03-27_12-19-25.csv завантажено
VHI_13_2025-03-27_12-19-26.csv завантажено
VHI_14_2025-03-27_12-19-27.csv завантажено
VHI_15_2025-03-27_12-19-28.csv завантажено
VHI_16_2025-03-27_12-19-29.csv завантажено
VHI_17_2025-03-27_12-19-30.csv завантажено
VHI_18_2025-03-27_12-19-31.csv завантажено
VHI_19_2025-03-27_12-19-32.csv завантажено
VHI_20_2025-03-27_12-19-33.csv завантажено
VHI_21_2025-03-27_12-19-34.csv завантажено
VHI_22_2025-03-27_12-19-35.csv завантажено
VHI_23_2025-03-27_12-19-36.csv завантажено
VHI_24_2025-03-27_12

Клінінг та обробка csv файлів у DataFrame

In [4]:
def cleaning(filepath):
    with open(filepath, "r", encoding="utf-8") as file:
        lines = [line.replace("</pre></tt>", "").strip("\n") for line in file]
    with open(filepath, "w", encoding="utf-8") as file:
        file.write("\n".join(lines))

def load_data(directory):
    all_data = []
    
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        
        cleaning(filepath) 
        
        with open(filepath, "r", encoding="utf-8") as file:
            first_line = file.readline().strip()
            region_name = first_line.split(":")[1].split(",")[0].strip()
        
        df = pd.read_csv(filepath, skiprows=2, header=0, usecols=[0, 1, 2, 3, 4, 5, 6], skip_blank_lines=True)
        df.columns = ["Year", "Week", "SMN", "SMT", "VCI", "TCI", "VHI"]
        
        region_index = filename.split("_")[1]
        df["Region"] = int(region_index)
        df["Region_Name"] = region_name
        df = df[df["VHI"] != -1]
        
        all_data.append(df)

    df = pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()
    df = df.dropna(subset=["Year"]) 
    return df

vhi = load_data(folder)

In [5]:
vhi.head()

,Year,Week,SMN,SMT,VCI,TCI,VHI,Region,Region_Name
0,1982,2,0.063,261.53,55.89,38.20,47.04,10,Khmel'nyts'kyy
1,1982,3,0.063,263.45,57.30,32.69,44.99,10,Khmel'nyts'kyy
2,1982,4,0.061,265.10,53.96,28.62,41.29,10,Khmel'nyts'kyy
3,1982,5,0.058,266.42,46.87,28.57,37.72,10,Khmel'nyts'kyy
4,1982,6,0.056,267.47,39.55,30.27,34.91,10,Khmel'nyts'kyy


Зміна індексів областей

In [6]:
def update_indx(df):
    pupu = {
        1:22, 2:24, 3:23, 4:25, 5:3, 6:4, 7:8,
        8:19, 9:20, 10:21, 11:9, 12:26,
        13:10, 14:11, 15:12, 16:13, 17:14,
        18:15, 19:16, 20:27, 21:17, 22:18,
        23:6, 24:1, 25:2, 26:7, 27:5
    }
    
    df["Region"] = df["Region"].map(pupu)
    return df
vhi = update_indx(vhi)

In [7]:
vhi.head()

,Year,Week,SMN,SMT,VCI,TCI,VHI,Region,Region_Name
0,1982,2,0.063,261.53,55.89,38.20,47.04,21,Khmel'nyts'kyy
1,1982,3,0.063,263.45,57.30,32.69,44.99,21,Khmel'nyts'kyy
2,1982,4,0.061,265.10,53.96,28.62,41.29,21,Khmel'nyts'kyy
3,1982,5,0.058,266.42,46.87,28.57,37.72,21,Khmel'nyts'kyy
4,1982,6,0.056,267.47,39.55,30.27,34.91,21,Khmel'nyts'kyy


In [8]:
vhi.to_csv("vhi_dataset.csv", index=False, encoding='utf-8')

1. Ряд vhi для області за певний рік

In [9]:
def vhi_reg_year(df, region_name, year):
    return df[(df['Region_Name'] == region_name) & (df['Year'] == year)][['Week', 'VHI']]

In [10]:
vhi_reg_year(vhi, "Kiev", 2017)

,Week,VHI
3954,1,41.44
3955,2,41.99
3956,3,45.06
3957,4,47.22
3958,5,46.35
3959,6,43.99
3960,7,40.00
3961,8,37.18
3962,9,36.84
3963,10,36.71


2. Пошук екстремумів (min та max) для вказаних областей та років, середнього, медіани

In [11]:
def extr(df, regions, years):
    subset = df[df["Region_Name"].isin(regions) & df["Year"].isin(years)]
    
    return subset.groupby(["Region_Name", "Year"])["VHI"].agg(
        Min_VHI="min",
        Max_VHI="max",
        Mean_VHI="mean",
        Median_VHI="median"
    ).reset_index()
    

In [12]:
extr(vhi, ["Kiev", "Poltava"], [2017, 2024])

,Region_Name,Year,Min_VHI,Max_VHI,Mean_VHI,Median_VHI
0,Kiev,2017,36.49,55.59,44.824423,43.81
1,Kiev,2024,33.41,55.40,43.957500,43.09
2,Poltava,2017,32.22,56.61,44.979615,46.10
3,Poltava,2024,23.90,57.31,41.902885,41.03


3. Ряд VHI за вказаний діапазон років для вказаних областей;

In [13]:
def vhi_range_year(df, regions, start_year, end_year, week):
    return df[(df["Region_Name"].isin(regions)) & (df["Year"].between(start_year, end_year)) & (df["Week"] == (week)) ][["Year", "Week", "Region_Name", "VHI"]]

In [14]:
vhi_range_year(vhi, ["Poltava", "Kiev"], 2023, 2024, 5)

,Year,Week,Region_Name,VHI
4270,2023,5,Kiev,48.43
4322,2024,5,Kiev,47.99
19565,2023,5,Poltava,49.77
19617,2024,5,Poltava,52.69


4. Для всього набору даних виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні (20% областей - 5 областей з 25). Повернути роки, назви областей з екстремальними посухами та значення VHI;

In [15]:
def vhi_dryness(vhi, min_reg, dryness=15):
    minimal = min_reg * 27 / 100

    ex_dryness = vhi[(vhi['VHI'] < dryness)]
    year_dryness = ex_dryness.groupby('Year')['Region_Name'].nunique()
    crit_year = year_dryness[year_dryness > minimal].index

    result = (ex_dryness[ex_dryness['Year'].isin(crit_year)]
              .groupby(['Year', 'Region_Name'])['VHI']
              .min()
              .reset_index())
    return result


In [16]:
vhi_dryness(vhi, 10)

,Year,Region_Name,VHI
0,2000,Cherkasy,10.68
1,2000,Kharkiv,9.36
2,2000,Kiev,10.60
3,2000,Kiev City,6.49
4,2000,Sevastopol',8.14
5,2000,Vinnytsya,11.25
6,2007,Crimea,13.28
7,2007,Kherson,12.23
8,2007,Mykolayiv,5.94
9,2007,Odessa,5.52
